In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import math
import time
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dtype_train = {
               'row_id': 'int64', 
               'timestamp': 'int64', 
               'user_id': 'int32', 
               'content_id': 'int32', 
               'content_type_id': 'int8',
               'task_container_id': 'int16', 
               'user_answer': 'int8', 
               'answered_correctly': 'int8', 
               'prior_question_elapsed_time': 'float32', 
               'prior_question_had_explanation': 'boolean',
                }
dtype_questions = {
                'question_id': 'int32',
                'part': 'int8',
                }

In [ ]:
train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', low_memory=False,skiprows= range(1, 9*(10**7)), 
                       dtype=dtype_train, usecols=dtype_train.keys()
                      )
print (train.shape)
train = train[train['content_type_id'] == 0]
questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv', dtype=dtype_questions, 
                          usecols=dtype_questions.keys(), index_col='question_id' )
train = train.join(questions, on = 'content_id')
cols = ['content_id', 'task_container_id', 'prior_question_had_explanation', 'part']
resp_cols = ['answered_correctly','timestamp', 'prior_question_elapsed_time']
cols_ = cols + resp_cols + ['user_id']
exclude_list = ['part', 'user_id']
train = train[cols_]

**Data preprocessing**
- Add 1 to all columns to reserve 0 for embedding 
- Compute lagtime: time between a user interactions
- Convert lagtime to minutes
- Convert prior_question_elapsed_time to minutes and cap it to 300
- Create a vocab_size dictionary to count the number of unique values for each column (we will need it for the embedding layers)

In [ ]:

def preprocess_data(train, exclude_list):
    for col in cols_:
        if col not in exclude_list:
            train[col] += 1
    train.fillna({'prior_question_elapsed_time' : 0, 'prior_question_had_explanation' : 4}, inplace=True)
    train['lagtime'] = train.groupby('user_id')['timestamp'].shift()
    train['lagtime']=train['timestamp']-train['lagtime']
    train['lagtime'].fillna(0, inplace=True)
    train.lagtime=train.lagtime.astype('int32')
    train['timestamp'] = train['lagtime']
    train.drop('lagtime', axis='columns', inplace=True)
    train['timestamp'] = train['timestamp'].apply(lambda x: x // 60000).astype('int32')
    train['prior_question_elapsed_time'] = train['prior_question_elapsed_time'].apply(lambda x: min(300, x / 1000))
preprocess_data(train,exclude_list )
vocab_size = {}
for col in ['prior_question_had_explanation', 'part', 'timestamp']:
    vocab_size[col] = int(train[col].max() + 2)

vocab_size['content_id'] = 30000
vocab_size['task_container_id'] = 20001


**Data Generator**

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, train, seq_len, cols,training, start_index):
        self.train = train
        self.train_keys = list(self.train.keys())
        self.cols = cols
        self.training = training
        self.seq_len = seq_len
        self.start_index = start_index
        self.user_index = dict(zip([i +  self.start_index for i in range(len(self.train))], self.train_keys))
        np.random.shuffle(self.train_keys)
        
    
    def __len__(self):
        return len(self.train)
    
    
    def rolling_window(self,a, w):
        s0, s1 = a.strides
        m, n = a.shape
        return np.lib.stride_tricks.as_strided(
            a, 
            shape=(m-w+1, w, n), 
            strides=(s0, s0, s1)
        )
    
    def get_sequences(self, x, resp, y):
        exercise = np.pad(x, [[ self.seq_len-1, 0], [0, 0]], constant_values=0)
        resp = np.concatenate((np.array([3.0, 3.0, 3.0]).reshape(1,3), resp[:- 1,:]))
        response = np.pad(resp, [[ self.seq_len-1, 0], [0, 0]], constant_values=0)
        exercise = self.rolling_window(exercise, self.seq_len)
        response = self.rolling_window(response, self.seq_len)
        if self.training:
            return exercise, response, y
        else:
                            
            return exercise, response
        
            
    
    def __getitem__(self, index):
        user = self.user_index[index +  self.start_index]
        user_df = self.train[user].values
        if len(user_df )> 150:
            seq_ind = np.random.randint(0,len(user_df) - 150 + 1)
            user_df = user_df[seq_ind:seq_ind + 150,:]
        x = user_df[:,:4]
        resp = user_df[:,4:7]
        y = np.array(user_df[:,4])
        if self.training:
            exercise, response, next_y = self.get_sequences(x,resp, y)
            return {'exercise': tf.convert_to_tensor(exercise, dtype=tf.float32),
                    'response' : tf.convert_to_tensor(response, dtype=tf.float32)
                   }, tf.convert_to_tensor(np.array(next_y), dtype=tf.int64)
        else:
            exercise, response = self.get_sequences(x,resp, y)
            return {'exercise': tf.convert_to_tensor(exercise, dtype=tf.float32),
                    'response' : tf.convert_to_tensor(response, dtype=tf.float32)
                   }
            
            

**Positional Encoding**

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, cols, vocab_size, d_model, max_len):
        super(PositionalEncoding, self).__init__()
        self.cols = cols
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_len = max_len
        self.embeddings = []
        for col in self.cols:
            self.embeddings.append(tf.keras.layers.Embedding(self.vocab_size[col],self.d_model))
    
    def call(self, x):
        embeddings = []
        for i in range(len(self.cols)):
            embeddings.append(self.embeddings[i](x[:,:,i]))
        return tf.keras.layers.Add()(embeddings)
        

def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights


class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)
        concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights
    
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-5)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):
        skip_x = x
        x = self.layernorm1(x)
        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        #attn_output = self.dropout1(attn_output, training=training)
        #out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
        out1 = skip_x + attn_output
        ffn_output = self.ffn(self.layernorm2(out1))
        out2 = out1 + ffn_output

        #ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        #ffn_output = self.dropout2(ffn_output, training=training)
        #out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2
    
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.layernorm4 = tf.keras.layers.LayerNormalization(epsilon=1e-5)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)


    def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
        # enc_output.shape == (batch_size, input_seq_len, d_model)
        x = self.layernorm1(x)
        enc_output = self.layernorm4(enc_output)

        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
        #attn1 = self.dropout1(attn1, training=training)
        out1 = x +  attn1
        skip_out = out1
        out1 = self.layernorm2(out1)
        

        attn2, attn_weights_block2 = self.mha2(
            enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
        #attn2 = self.dropout2(attn2, training=training)
        out2 = skip_out + attn2  # (batch_size, target_seq_len, d_model)
        ffn_output = self.ffn(self.layernorm3(out2))  # (batch_size, target_seq_len, d_model)
        #ffn_output = self.dropout3(ffn_output, training=training)
        out3 = out2 + ffn_output  # (batch_size, target_seq_len, d_model)
        

        return out3, attn_weights_block1, attn_weights_block2
    
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
               maximum_position_encoding, cols, vocab_dict, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.cols = cols
        self.vocab_dict = vocab_dict

        self.embedding = PositionalEncoding(cols, vocab_dict, d_model, maximum_position_encoding )
        self.pos_encoding = tf.keras.layers.Embedding(config['seq_len'], d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):
        
        #seq_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=config['seq_len'], delta=1)

        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        #x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding(positions)
        
        #x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)
    
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.dense_timestamp = tf.keras.layers.Embedding(35793,d_model) 
        self.dense_elapsed = tf.keras.layers.Embedding(302,d_model)
        #self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
        self.pos_encoding = tf.keras.layers.Embedding(config['seq_len'], d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
        #seq_len = tf.shape(x)[-1]
        attention_weights = {}
        positions = tf.range(start=0, limit=config['seq_len'], delta=1)
        x = self.embedding(x[:,:,0])  # (batch_size, target_seq_len, d_model)
        timestamp = self.dense_timestamp(x[:,:,1])
        elapsed_time = self.dense_elapsed(x[:,:,2])
        x = tf.keras.layers.Add()([x,timestamp, elapsed_time])
        #x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        #x += self.pos_encoding[:, :seq_len, :]
        x += self.pos_encoding(positions)
        #x = self.dropout(x, training=training)
        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights
    
    
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, 
               target_vocab_size, pe_input, pe_target, cols, vocab_dict, rate=0.1):
        super(Transformer, self).__init__()
        
        self.cols = cols
        self.vocab_dict = vocab_dict
        self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                            pe_input,cols, vocab_dict, rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                           target_vocab_size, pe_target, rate)
        #self.final_layer = tf.keras.layers.Dense(1, activation = 'softmax')
        #self.output_layer = tf.keras.layers.TimeDistributed(self.final_layer)
        #self.global_average = tf.keras.layers.GlobalAveragePooling1D()
        #self.dropout1 = tf.keras.layers.Dropout(0.1)
        self.final_layer = tf.keras.layers.Dense(1, activation='linear')
        #self.dropout2 = tf.keras.layers.Dropout(0.1)
        #self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')
        #self.output_layer = tf.keras.layers.Softmax()

    def call(self, inp, tar, training, enc_padding_mask, dec_padding_mask):

        enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)
        

        # dec_output.shape == (batch_size, tar_seq_len, d_model)
        dec_output, attention_weights = self.decoder(
            tar, enc_output, training, enc_padding_mask, dec_padding_mask)
        #global_average = self.global_average(dec_output)
        #dropout1 = self.dropout1(global_average)
        #final_output = self.final_layer(dec_output)
        #dropout2 = self.dropout2(final_output)
        

        final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)
        #output_l = self.output_layer(dec_output)
        

        return final_output, attention_weights

In [ ]:
config = {
    'seq_len' : 70,
    'batch_size' : 512
}
params = {
    'num_layers' : 2,
    'd_model' : 128,
    'num_heads' : 4,
    'dff' : 512,
    'target_vocab_size' : 4,
    'pe_input' : config['seq_len'] + 2,
    'pe_target' : config['seq_len'] + 2,
    'cols' : cols,
    'vocab_dict' : vocab_size
}

In [ ]:
gc.collect()

In [ ]:
def create_padding_mask(seq):
    seq = tf.cast(tf.reduce_all(tf.math.equal(seq, 0),-1), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)
def create_masks(inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)
    dec_target_padding_mask = create_padding_mask(tar)
    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    #dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by 
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(config['seq_len'])
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
    combined_encoder_mask = tf.maximum(enc_padding_mask, look_ahead_mask)
    return combined_encoder_mask, combined_mask

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=500):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)/100
learning_rate = CustomSchedule(params['d_model'])
optimizer = tf.keras.optimizers.Adam(learning_rate =learning_rate, beta_1=0.9, beta_2=0.999, 
                                     epsilon=1e-8)
#optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4)
loss_object = tf.keras.losses.binary_crossentropy
auc = tf.keras.metrics.AUC( name='AUC')
bin_acc = tf.keras.metrics.BinaryAccuracy(name='acc')
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')
train_auc = tf.keras.metrics.Mean(name='train_auc')
val_auc = tf.keras.metrics.AUC( name='val_AUC')
val_bin_acc = tf.keras.metrics.BinaryAccuracy(name='val_acc')
val_loss = tf.keras.metrics.Mean(name='val_loss')
validation_accuracy = tf.keras.metrics.Mean(name='val_accuracy')
validation_auc = tf.keras.metrics.Mean(name='val_auc')

In [ ]:
import matplotlib.pyplot  as plt
temp_learning_rate_schedule = CustomSchedule(params['d_model'])
plt.plot(temp_learning_rate_schedule(tf.range(3500, dtype=tf.float32)))
plt.ylabel("Learning Rate")
plt.xlabel("Train Step")

In [ ]:
def loss_function(real, pred):
    real = real - 1
    real = real[:, np.newaxis]
    loss_ = loss_object(real, pred)


def accuracy_function(y_true, y_pred, bin_acc):
    y_true = y_true - 1
    y_true = y_true[:, np.newaxis]
    bin_acc.update_state(y_true, y_pred)
    acc = bin_acc.result()
    return tf.math.reduce_mean(acc)
def auc_function(y_true, y_pred, auc):
    
    y_true = y_true - 1
    y_true = y_true[:, np.newaxis]
    auc.update_state(y_true, y_pred)
    auc_ = auc.result()
    return tf.math.reduce_mean(auc_)


In [ ]:

train_step_signature = [
    {'exercise': tf.TensorSpec(shape=(None,None,None), dtype=tf.float32),
      'response':tf.TensorSpec(shape=(None,None,None), dtype=tf.float32)},
    tf.TensorSpec(shape=(None,  ), dtype=tf.int64),
                           ]

transformer = Transformer(**params)
transformer.load_weights('../input/checkpoints4/my_checkpoint')
    
@tf.function(input_signature=train_step_signature)
def train_step(x, y):
    exercise = x['exercise']
    response = x['response']
    enc_padding_mask, dec_padding_mask = create_masks(exercise, response)
    with tf.GradientTape() as tape:
        predictions, _ = transformer(exercise, response, 
                                True, 
                                enc_padding_mask, 
                                dec_padding_mask)
        y = y - 1
        y = y[:, np.newaxis]
        predictions = tf.math.sigmoid(predictions)
        predictions = predictions[:,-1,:]
        loss = loss_object(y , predictions)
    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
    bin_acc.update_state(y , predictions)
    auc.update_state(y, predictions)
    train_loss.update_state(loss)
   
        
@tf.function(input_signature=train_step_signature)
def test_step(x, y):
    exercise = x['exercise']
    response = x['response']
    enc_padding_mask, dec_padding_mask = create_masks(exercise, response)
    predictions, _ = transformer(exercise, response, 
                                False, 
                                enc_padding_mask, 
                                dec_padding_mask)
    predictions =tf.math.sigmoid(predictions)
    predictions = predictions[:,-1,:]
    y = y - 1
    y = y[:, np.newaxis]
    loss = loss_object(y , predictions)
    val_bin_acc.update_state(y , predictions)
    val_auc.update_state(y, predictions)
    val_loss.update_state(loss)
    

    
    
    

In [ ]:
train_idx = int(len(train)*0.8)
train_df = train[:train_idx]
train_group = dict(tuple(train_df.groupby('user_id')))
del train_df
gc.collect()
val_df = train[train_idx:]
val_group = dict(tuple(val_df.groupby('user_id')))
del val_df
gc.collect()

In [ ]:

def update_train_states():
    train_loss.reset_states()
    bin_acc.reset_states()
    auc.reset_states()
def update_val_states():
    val_loss.reset_states()
    val_bin_acc.reset_states()
    val_auc.reset_states()
    
for epoch in range(3):
    print (f'****************************Epoch {epoch}************************************** ')
    train_iter = DataGenerator2(train_group,config['seq_len'], cols, True,0)
    steps = int(len(train_iter) )
    print('training steps', steps)
    count = 0
    countb = 0
    exercises = []
    responses = []
    ys = []
    for batch, (x,y) in enumerate(train_iter):
        exercises.extend(x['exercise'])
        responses.extend(x['response'])
        ys.extend(y)
        countb += y.get_shape().as_list()[0]
        if countb >= config['batch_size']:
            (x_,y_) = {'exercise': tf.convert_to_tensor(exercises, dtype=tf.float32),
                    'response' : tf.convert_to_tensor(responses, dtype=tf.float32)
                    }, tf.convert_to_tensor(ys, dtype=tf.int64)
            countb = 0
            exercises = []
            responses = []
            ys = []
            train_step(x_,y_)
            if count % 100 ==0:
                print ('Batch {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
                     batch, train_loss.result(), bin_acc.result(), auc.result()))
            count += 1
            if count > steps:
                break
   
    update_train_states()
    print ('validation')
    val_iter = DataGenerator2(val_group,config['seq_len'], cols, True,train_idx)
    val_steps =int(len(val_iter) )
    print('validation steps', val_steps)
    auc_score = []
    valcount = 0
    countb = 0
    exercises = []
    responses = []
    ys = []
    for batch, (x,y) in enumerate(val_iter): 
        exercises.extend(x['exercise'])
        responses.extend(x['response'])
        ys.extend(y)
        countb += y.get_shape().as_list()[0]
        if countb >= config['batch_size'] or countb >= val_steps:
            (x_,y_) = {'exercise': tf.convert_to_tensor(exercises, dtype=tf.float32),
                    'response' : tf.convert_to_tensor(responses, dtype=tf.float32)
                    }, tf.convert_to_tensor(ys, dtype=tf.int64)
            countb = 0
            exercises = []
            responses = []
            ys = []
            test_step(x_, y_)
            auc_score.append(val_auc.result())
        if valcount % 100 ==0:
            print ('Batch {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
                 batch, val_loss.result(), val_bin_acc.result(), val_auc.result()))
        valcount += 1
        if valcount > val_steps:
            break
    print ('validation score', np.array(auc_score).mean())
    update_val_states()
        
        
    
    

In [ ]:
transformer.save_weights('./checkpoints/my_checkpoint')

In [ ]:
train_group = train.groupby('user_id')
train_dict = dict(tuple(train_group))
columns = cols + ['timestamp', 'prior_question_elapsed_time']+ ['user_id', 'row_id']
del train

In [ ]:
def update_train(prior_test):
    #prior_test['answered_correctly'] = np.array(prior_correct)
    #train_df = pd.concat([train_df, prior_test])
    test_group = dict(tuple(prior_test.groupby('user_id')))
    for user in test_group.keys():
        if user in train_dict.keys():
            train_dict[user] = pd.concat([train_dict[user],test_group[user]]).tail(config['seq_len'])
        else:
            train_dict[user] = test_group[user].tail(config['seq_len'])    
            
    #user_dict.update(test_group.size().to_dict())
    #train_dict = train_group.size().to_dict()
    #return train_df, train_group
def preprocess_test(train, exclude_list):
    for col in cols:
        if col not in exclude_list:
            train[col] += 1
    train.fillna({'prior_question_elapsed_time' : 0, 'prior_question_had_explanation' : 4}, inplace=True)
    train['lagtime'] = train.groupby('user_id')['timestamp'].shift()
    train['lagtime']=train['timestamp']-train['lagtime']
    train['lagtime'].fillna(0, inplace=True)
    train.lagtime=train.lagtime.astype('int32')
    train['timestamp'] = train['lagtime']
    train.drop('lagtime', axis='columns', inplace=True)
    # convert timestamp to number of days and cap the number to 600
    #train['timestamp'] = train['timestamp'].apply(lambda x: min(600, max(x / 86400000,0)))
    train['timestamp'] = train['timestamp'].apply(lambda x: min(x // 60000, 35790)).astype('int32')
    #max_time_stamp = max(train['timestamp'].values.max(),1)
    #train['timestamp'] /=max_time_stamp
    #convert prior_question_elapsed_time to seconds and cap the max to 300
    train['prior_question_elapsed_time'] = train['prior_question_elapsed_time'].apply(lambda x: min(300, x / 1000))
    
    
def rolling_window(a, w):
        s0, s1 = a.strides
        m, n = a.shape
        return np.lib.stride_tricks.as_strided(
            a, 
            shape=(m-w+1, w, n), 
            strides=(s0, s0, s1)
        )

def generate_sequence(row, seq_len = config['seq_len']):
    #user  = row['user_id']
    user = row[-1]
    if user  in train_dict.keys():
        data = train_dict[user].tail(seq_len - 1).values
        pad_size =  seq_len - len(data) - 1
        x = np.pad(np.concatenate([data[:,:4],row[:4].reshape(1,4)]), [[ pad_size , 0], [0, 0]], constant_values=0)
        x = rolling_window(x, seq_len) 
        y = np.concatenate((np.array([3.0, 3.0, 3.0]).reshape(1,3), data[:,4:7]))
        y = np.pad(y, [[ pad_size , 0], [0, 0]], constant_values=0)     
        y = rolling_window(y, seq_len) 
    else:
        x = np.pad(row[:4].reshape(1,4), [[ seq_len - 1 , 0], [0, 0]], constant_values=0)
        x = rolling_window(x, seq_len)
        y = np.concatenate([[0,0,0] * (seq_len - 1),[3.,3.,3.]]).reshape(seq_len,3)
        y = rolling_window(y, seq_len) 
    return {'exercise':x, 'response':y }  
    #return item

In [ ]:
predict_step_signature = {'exercise': tf.TensorSpec(shape=(None,None,None), dtype=tf.float32),
      'response':tf.TensorSpec(shape=(None,None,None), dtype=tf.float32)}
                           
@tf.function(input_signature=[predict_step_signature])
#@tf.function
def predict_step(x):
    exercise = x['exercise']
    response = x['response']
    enc_padding_mask, dec_padding_mask = create_masks(exercise, response)
    predictions, _ = transformer(exercise, response, 
                                False, 
                                enc_padding_mask, 
                                dec_padding_mask)
    predictions =tf.math.sigmoid(predictions)
    predictions = predictions[:,-1,:]
    return predictions

In [ ]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
for test, sample_prediction in iter_test:
   
    
    try:
        prior_correct = eval(test['prior_group_answers_correct'].iloc[0])
        prior_correct = [a +1   for a in prior_correct if a != -1]
    except:
        prior_correct = []
    
    # Add prior correct to test and update stored users
    if prior_correct:
        prior_test.insert(4,'answered_correctly', prior_correct)
        #prior_test['answered_correctly'] = prior_correct
        update_train(prior_test)
        

    # Filter test
    test = test.loc[
        test['content_type_id'] == 0]
    # add questions
    test = test.join(questions, on = 'content_id')
    test = test[columns]

    # Add global features
    preprocess_test(test, exclude_list)

    # Save test for later
    prior_test = test.drop(columns='row_id').copy()

    # Make x
    sequences = np.apply_along_axis(
        generate_sequence,
        1,
        test.drop(columns='row_id').to_numpy()
    )
    x = [np.squeeze(s['exercise'], axis = 0) for s in sequences]
    y = [np.squeeze(s['response'], axis = 0) for s in sequences]
    items = {'exercise': tf.convert_to_tensor(x, dtype=tf.float32),
                'response' : tf.convert_to_tensor(y, dtype=tf.float32)
                }
     # Predict
    predictions = predict_step(items)
    test['answered_correctly'] = predictions.numpy()

    
    env.predict(test[['row_id', 'answered_correctly']])